In [1]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [3]:
# m = yaml.safe_load(open('/diskmnt/Projects/Users/estorrs/pecgs-pipeline/examples/pecgs_TN_wxs_bam//test_sample/C3L-00677/summary.yaml'))
# m

In [4]:
template = yaml.safe_load(open('../cwl/template.cnv.yaml'))
template

{'common_biallelic': {'class': 'File', 'path': 'a/file/path'},
 'normal_bam': {'class': 'File', 'path': 'a/file/path'},
 'pool_of_normals': {'class': 'File', 'path': 'a/file/path'},
 'protein_coding_gene': {'class': 'File', 'path': 'a/file/path'},
 'reference_dir': {'class': 'Directory', 'path': 'a/file/path'},
 'sample': 'a_string',
 'target_interval_list': {'class': 'File', 'path': 'a/file/path'},
 'tumor_bam': {'class': 'File', 'path': 'a/file/path'}}

In [9]:
template = yaml.safe_load(open('/diskmnt/Projects/Users/estorrs/wombat/tests/data/cnv/C3L-00677_inputs.yaml'))
template

{'common_biallelic': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'},
 'cytoband': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/cytoBand.txt'},
 'normal_bam': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'},
 'pool_of_normals': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/create_pon/gatk4scnaPON.Normal.hdf5'},
 'protein_coding_gene': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/gencode.v34.annotation.gene_filtered.protein_coding.ensembl_ID_no_version.protein-coding_hgnc_filtered.duplicates_removed.ensembl_ID_removed.txt'},
 'reference_dir': {'class': 'Directory',
  'path': '/storag

In [7]:
# template['common_biallelic']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf'
# template['reference_dir']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/references/GRCh38.d1.vd1'
# template['pool_of_normals']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/pon/pecgs_pon_v1/create_pon/gatk4scnaPON.Normal.hdf5'
# template['target_interval_list']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.autosome_only.bed.preprocessed.exome.interval_list'
# template['protein_coding_gene']['path'] = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/gencode.v34.annotation.gene_filtered.protein_coding.ensembl_ID_no_version.protein-coding_hgnc_filtered.duplicates_removed.ensembl_ID_removed.txt'
# template['cytoband'] = {
#     'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/cnv/db/cytoBand.txt',
#     'class': 'File'}


In [8]:
# yaml.safe_dump(template, open('/diskmnt/Projects/Users/estorrs/wombat/tests/data/cnv/C3L-00677_inputs.yaml', 'w'))

In [9]:
resource_dir = '/data/pecgs/cnv'
data_dir = '/data/pecgs/test_sample/C3L-00677'
output_dir = '/data/pecgs/test_runs/cnv/' + m['id']
Path(output_dir).mkdir(parents=True, exist_ok=True)
pieces = [
    'docker', 'run',
    '-v', f'{resource_dir}:/cnv_resources',
    '-v', f'{data_dir}:/cnv_data',
    '-v', f'{output_dir}:/cnv_outputs',
    '-it', 'estorrs/pecgs_cnv:0.0.2',
    '/bin/bash'
]
cmd = ' '.join(pieces)
cmd

'docker run -v /data/pecgs/cnv:/cnv_resources -v /data/pecgs/test_sample/C3L-00677:/cnv_data -v /data/pecgs/test_runs/cnv/C3L-00677:/cnv_outputs -it estorrs/pecgs_cnv:0.0.1 /bin/bash'

In [14]:
pieces = [f'python /pecgs-cnv/cnv/cnv.py',
         '--common-biallelic', '/cnv_resources/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf',
          '--genome', '/cnv_resources/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa',
          '--genome-dict', '/cnv_resources/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict',
          '--pool-of-normals', '/cnv_resources/pon/create_pon/gatk4scnaPON.Normal.hdf5',
          '--protein-coding-gene', '/cnv_resources/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed',
          '--target-interval-list', '/cnv_resources/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list',
          '--gene-level-script', '/pecgs-cnv/cnv/segment_to_geneLevel.py',
          '--merge-gene-script', '/pecgs-cnv/cnv/mergeMultipleFilesToOne.py',
          '--out-dir', 'outputs',
          m['id'],
          '/cnv_data/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam',
          '/cnv_data/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'
         ]
cmd = ' '.join(pieces)
cmd






'python /pecgs-cnv/cnv/cnv.py --common-biallelic /cnv_resources/db/snp_database/af-only-gnomad.hg38.common_biallelic.chr1-22XY.vcf --genome /cnv_resources/references/GRCh38.d1.vd1/GRCh38.d1.vd1.fa --genome-dict /cnv_resources/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict --pool-of-normals /cnv_resources/pon/create_pon/gatk4scnaPON.Normal.hdf5 --protein-coding-gene /cnv_resources/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed --target-interval-list /cnv_resources/db/IDT_xGen_Exome_Research_Panel_v1.hg38.removed_alt_chr.bed.target.preprocessed.exome.interval_list --gene-level-script /pecgs-cnv/cnv/segment_to_geneLevel.py --merge-gene-script /pecgs-cnv/cnv/mergeMultipleFilesToOne.py --out-dir outputs C3L-00677 /cnv_data/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam /cnv_data/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'

In [15]:
# gatk PlotDenoisedCopyRatios --standardized-copy-ratios outputs/C3L-00677.T.standardizedCR.tsv --denoised-copy-ratios outputs/C3L-00677.T.denoisedCR.tsv --sequence-dictionary /cnv_resources/references/GRCh38.d1.vd1/GRCh38.d1.vd1.dict --minimum-contig-length 10000000 --output outputs/plots --output-prefix C3L-00677.T --tmp-dir outputs


In [ ]:
# python /pecgs-cnv/cnv/segment_to_geneLevel.py --prefix C3L-00677.T --name C3L-00677 --seg outputs/C3L-00677.T.called.igv.seg --gene /cnv_resources/db/remaking_protein-coding/gencode.v34.annotation.gene_filterd.need_gene_symbol.no_sym.filtered_to_hgnc_protein-coding_genes.bed -o outputs
